In [67]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [69]:
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")

In [71]:
train.head()

CustomerID  Age  Gender  AnnualIncome  SpendingScore  YearsAsCustomer  \
0           1   56    Male           NaN      66.978399               11   
1           2   69  Female  60275.627399      77.993229               15   
2           3   46  Female  59795.569870      90.811984                3   
3           4   32    Male  64889.496054      69.632267               13   
4           5   60    Male  28388.877057      36.000510                4   

   NumProductsPurchased  WebsiteVisitsPerMonth  LastPurchaseAmount  TimeOnApp  \
0                     1                      3          201.860693   2.902409   
1                     8                      4          446.681209   1.196631   
2                     9                     27          622.056365        NaN   
3                     9                      7          875.767959   8.067661   
4                     8                      3          476.568467   9.085315   

   ...  CustomerSatisfaction  PreferredDevice Region IsPremiumMember  \
0  ...                   1.0           Tablet    NaN               1   
1  ...                   4.0           Mobile   East               0   
2  ...                   5.0              NaN  North               0   
3  ...                   4.0           Mobile  North               1   
4  ...                   2.0              NaN   West               0   

   ReceivedPromotions  ClickedAds  ReturnedItems  UsedCoupons  FeedbackScore  \
0                   3           6              4            0              6   
1                   2           0              2            1              8   
2                   3           1              0            1              3   
3                   0           7              2            2             10   
4                   4           9              4            0              7   

   Churn  
0      0  
1      0  
2      0  
3      0  
4      0  

[5 rows x 21 columns]

In [73]:
train.shape

(5000, 21)

In [75]:
test.shape

(200, 20)

In [77]:
train.columns

Index(['CustomerID', 'Age', 'Gender', 'AnnualIncome', 'SpendingScore',
       'YearsAsCustomer', 'NumProductsPurchased', 'WebsiteVisitsPerMonth',
       'LastPurchaseAmount', 'TimeOnApp', 'TimeOnWebsite',
       'CustomerSatisfaction', 'PreferredDevice', 'Region', 'IsPremiumMember',
       'ReceivedPromotions', 'ClickedAds', 'ReturnedItems', 'UsedCoupons',
       'FeedbackScore', 'Churn'],
      dtype='object')

In [79]:
# Drop the Customer ID
train.drop("CustomerID",axis=1,inplace=True)
test.drop("CustomerID",axis=1,inplace=True)

In [81]:
# summary
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    5000 non-null   int64  
 1   Gender                 5000 non-null   object 
 2   AnnualIncome           4500 non-null   float64
 3   SpendingScore          5000 non-null   float64
 4   YearsAsCustomer        5000 non-null   int64  
 5   NumProductsPurchased   5000 non-null   int64  
 6   WebsiteVisitsPerMonth  5000 non-null   int64  
 7   LastPurchaseAmount     5000 non-null   float64
 8   TimeOnApp              4500 non-null   float64
 9   TimeOnWebsite          5000 non-null   float64
 10  CustomerSatisfaction   4500 non-null   float64
 11  PreferredDevice        4500 non-null   object 
 12  Region                 4500 non-null   object 
 13  IsPremiumMember        5000 non-null   int64  
 14  ReceivedPromotions     5000 non-null   int64  
 15  Clic

In [83]:
# Duplicates
train = train.drop_duplicates()

In [85]:
# Handling the missing values
train.isna().sum()

Age                        0
Gender                     0
AnnualIncome             500
SpendingScore              0
YearsAsCustomer            0
NumProductsPurchased       0
WebsiteVisitsPerMonth      0
LastPurchaseAmount         0
TimeOnApp                500
TimeOnWebsite              0
CustomerSatisfaction     500
PreferredDevice          500
Region                   500
IsPremiumMember            0
ReceivedPromotions         0
ClickedAds                 0
ReturnedItems              0
UsedCoupons                0
FeedbackScore              0
Churn                      0
dtype: int64

In [87]:
# Imputation

# continous variables
train['AnnualIncome'] = train['AnnualIncome'].fillna(train['AnnualIncome'].mean())
train['TimeOnApp'] = train['TimeOnApp'].fillna(train['TimeOnApp'].mean())
train['CustomerSatisfaction'] = train['CustomerSatisfaction'].fillna(train['CustomerSatisfaction'].median())

# categorical variables
train['PreferredDevice'] = train['PreferredDevice'].fillna(train['PreferredDevice'].mode().iloc[0])
train['Region'] = train['Region'].fillna(train['Region'].mode().iloc[0])

In [89]:
train.isna().sum()

Age                      0
Gender                   0
AnnualIncome             0
SpendingScore            0
YearsAsCustomer          0
NumProductsPurchased     0
WebsiteVisitsPerMonth    0
LastPurchaseAmount       0
TimeOnApp                0
TimeOnWebsite            0
CustomerSatisfaction     0
PreferredDevice          0
Region                   0
IsPremiumMember          0
ReceivedPromotions       0
ClickedAds               0
ReturnedItems            0
UsedCoupons              0
FeedbackScore            0
Churn                    0
dtype: int64

In [91]:
train['Churn'].value_counts()

Churn
0    4771
1     229
Name: count, dtype: int64

In [99]:
train_x = train.drop(["Churn"], axis=1)
train_y = train["Churn"]

In [103]:
categorical_cols = train_x.select_dtypes(include=["object"]).columns
continous_cols = train_x.select_dtypes(include=["int64","float64"]).columns

In [113]:
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(train_x[categorical_cols])
encoded_df = pd.DataFrame(encoded_data.toarray(), columns= encoder.get_feature_names_out(categorical_cols))

scaler = StandardScaler()
scaled_data = scaler.fit_transform(train_x[continous_cols])
scaled_df = pd.DataFrame(scaled_data, columns=continous_cols)

train_x = pd.concat([encoded_df.reset_index(drop=True), scaled_df.reset_index(drop=True)], axis = 1)

In [115]:
train_x

Gender_Female  Gender_Male  PreferredDevice_Desktop  \
0               0.0          1.0                      0.0   
1               1.0          0.0                      0.0   
2               1.0          0.0                      0.0   
3               0.0          1.0                      0.0   
4               0.0          1.0                      0.0   
...             ...          ...                      ...   
4995            0.0          1.0                      1.0   
4996            1.0          0.0                      0.0   
4997            1.0          0.0                      0.0   
4998            1.0          0.0                      0.0   
4999            1.0          0.0                      0.0   

      PreferredDevice_Mobile  PreferredDevice_Tablet  Region_East  \
0                        0.0                     1.0          0.0   
1                        1.0                     0.0          1.0   
2                        1.0                     0.0          0.0   
3                        1.0                     0.0          0.0   
4                        1.0                     0.0          0.0   
...                      ...                     ...          ...   
4995                     0.0                     0.0          0.0   
4996                     0.0                     1.0          1.0   
4997                     1.0                     0.0          0.0   
4998                     1.0                     0.0          1.0   
4999                     1.0                     0.0          0.0   

      Region_North  Region_South  Region_West       Age  ...  \
0              0.0           0.0          1.0  0.832265  ...   
1              0.0           0.0          0.0  1.703719  ...   
2              1.0           0.0          0.0  0.161916  ...   
3              1.0           0.0          0.0 -0.776573  ...   
4              0.0           0.0          1.0  1.100405  ...   
...            ...           ...          ...       ...  ...   
4995           0.0           0.0          1.0 -1.312852  ...   
4996           0.0           0.0          0.0  1.502614  ...   
4997           1.0           0.0          0.0 -1.178782  ...   
4998           0.0           0.0          0.0  0.631160  ...   
4999           0.0           0.0          1.0 -0.508433  ...   

      LastPurchaseAmount     TimeOnApp  TimeOnWebsite  CustomerSatisfaction  \
0              -1.039965 -8.946194e-01      -0.840869             -1.494825   
1              -0.180748 -1.550845e+00      -1.141734              0.727412   
2               0.434746  3.416892e-16      -0.409507              1.468158   
3               1.325168  1.092493e+00       0.101603              0.727412   
4              -0.075856  1.483993e+00      -1.384602             -0.754079   
...                  ...           ...            ...                   ...   
4995            0.124061  1.746555e+00       0.181350             -0.013333   
4996            0.546193  1.504631e+00       0.431468              0.727412   
4997           -1.564830  3.416892e-16       1.239818             -0.754079   
4998           -0.757886 -1.261973e+00      -0.054267             -1.494825   
4999           -0.002836  9.068193e-01       0.500341              1.468158   

      IsPremiumMember  ReceivedPromotions  ClickedAds  ReturnedItems  \
0            0.990446            0.711569    0.493965       1.413577   
1           -1.009647            0.004244   -1.594683      -0.000141   
2           -1.009647            0.711569   -1.246575      -1.413860   
3            0.990446           -1.410407    0.842073      -0.000141   
4           -1.009647            1.418895    1.538290       1.413577   
...               ...                 ...         ...            ...   
4995         0.990446            0.711569   -0.550359      -1.413860   
4996        -1.009647            0.711569   -1.594683       1.413577   
4997         0.990446            1.418895    1.190182      -0.000141   
4998  

In [117]:
# Class Balance

# SMOTE
from collections import Counter
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_sampled, y_sampled = smote.fit_resample(train_x, train_y)

print("SMOTE class distribution :",Counter(y_sampled))

SMOTE class distribution : Counter({0: 4771, 1: 4771})


In [119]:
# continous variables
test['AnnualIncome'] = test['AnnualIncome'].fillna(test['AnnualIncome'].mean())
test['TimeOnApp'] = test['TimeOnApp'].fillna(test['TimeOnApp'].mean())
test['CustomerSatisfaction'] = test['CustomerSatisfaction'].fillna(test['CustomerSatisfaction'].median())

# categorical variables
test['PreferredDevice'] = test['PreferredDevice'].fillna(test['PreferredDevice'].mode().iloc[0])
test['Region'] = test['Region'].fillna(test['Region'].mode().iloc[0])

In [129]:
encoded_data = encoder.transform(test[categorical_cols])
encoded_df = pd.DataFrame(encoded_data.toarray(), columns= encoder.get_feature_names_out(categorical_cols))

scaled_data = scaler.transform(test[continous_cols])
scaled_df = pd.DataFrame(scaled_data, columns=continous_cols)

test_x = pd.concat([encoded_df.reset_index(drop=True), scaled_df.reset_index(drop=True)], axis = 1)

In [121]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(X_sampled, y_sampled)



SVC(kernel='linear', random_state=42)

In [135]:
y_pred = svm_classifier.predict(test_x)

In [137]:
y_pred

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0], dtype=int64)

In [141]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv("submit.csv")